In [435]:
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import requests
import random
import csv
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os

ON CHARGE LES LISTES DES LIGNES ET DES ARRETS


In [425]:
df_lignes = pd.read_csv('src/referentiel-des-lignes.csv', sep=';', index_col='ID_Line')
df_arrets = pd.read_csv('src/arrets.csv', sep=';', index_col='ArRId')

LES DONNEES QU'ON VEUT UTILISER POUR L'INSTANT

In [426]:
ar_prof_vaillant = '21611'
ar_bondy_rer = '41013'
ar_fdc = '463048'
ar_ob = '463059'

li_E = 'C01729'
li_105 = 'C01134'

# ON GARDE UNE LISTE DES INFOS RECUPEREES 
infos_messages = []
infos_horaires = []

FONCTIONS DE RECHERCHE DES INFOS D'UNE LIGNE ET D'UN ARRET

In [427]:
def get_line_name(s):
    line = s[-7:-1]
    if (pd.isnull(df_lignes.loc[line]['NetworkName'])):
         ret = df_lignes.loc[line]['TransportMode'] +' '+f"{df_lignes.loc[line]['ShortName_Line']}"
    else: 
        ret = df_lignes.loc[line]['NetworkName']+' '+f"{df_lignes.loc[line]['ShortName_Line']}"
        
    return ret

def get_stop_name(s):
    arret = s[-6:-1]
    nom = df_arrets.loc[int(arret)]['ArRName']
    return nom

"MESSAGES AFFICHES SUR LES ECRANS"

In [440]:
url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'StopPointRef' : f'STIF:StopPoint:Q:{ar_bondy_rer}:'}

response = requests.get(url, headers=headers, params=p)

In [441]:
arret = response.json()['Siri']['ServiceDelivery']['GeneralMessageDelivery'][0]['InfoMessage'][0]['Content']['StopPointRef'][0]['value']
list_messages = response.json().get("Siri").get("ServiceDelivery").get('GeneralMessageDelivery')[0].get('InfoMessage')

for i in list_messages:
    arret_name = get_stop_name(arret)
    type_message = i['InfoChannelRef']['value']
    message = i.get('Content').get('Message')[0].get('MessageText').get('value')
    sysdate = datetime.datetime.strftime(datetime.datetime.now(),"%Y-%m-%d %H:%M:%S")
 
    infos_messages.append({"arret":arret_name, "message": message, "type_message":type_message,"sysdate": sysdate})
    #print(message)

In [442]:
infos_messages

[{'arret': 'Gare de Bondy',
  'message': 'Pour votre sécurité, nous vous invitons à ne laisser aucun bagage sans surveillance. Veuillez nous signaler tout colis ou bagage qui vous paraitrait abandonné.Merci de votre vigilance',
  'type_message': 'Information',
  'sysdate': '2024-11-24 15:58:00'},
 {'arret': 'Gare de Bondy',
  'message': "23-24/11 jusqu'à 16H : Rosa Parks non desservie dans les 2 sens et Pantin non desservie en direction de Paris. Motif : Travaux.",
  'type_message': 'Perturbation',
  'sysdate': '2024-11-24 15:58:00'},
 {'arret': 'Gare de Bondy',
  'message': "23/11-24/11 jusqu'à 16H : Les trains de l'axe Chelles et Tournan sont au départ et à l'arrivée de Paris Est. Motif : travaux",
  'type_message': 'Perturbation',
  'sysdate': '2024-11-24 15:58:00'},
 {'arret': 'Gare de Bondy',
  'message': 'Pour votre sécurité, nous vous invitons à ne laisser aucun bagage sans surveillance. Veuillez nous signaler tout colis ou bagage qui vous paraitrait abandonné.Merci de votre vig

PROCHAINS PASSAGES

In [431]:

url = "https://prim.iledefrance-mobilites.fr/marketplace/stop-monitoring?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'MonitoringRef' : f'STIF:StopPoint:Q:{ar_prof_vaillant}:'}


response = requests.get(url, headers=headers, params=p)

In [ ]:
response.json()

ON RECUPERE LE NOM DE L'ARRET ET LES DIFFERENTES DESTINATIONS

In [432]:
arret = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'][0]['value']
direction = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['DestinationName'][0]['value']


dir_names = []
directions = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

for d in directions:
    dir_name =d['MonitoredVehicleJourney']['DestinationName'][0]['value']
    if dir_name not in dir_names:
        dir_names.append(dir_name)

print(f"Arrêt : {arret}")
print(f"Directions : {dir_names}")

Arrêt : Professeur Vaillant
Directions : ['Porte des Lilas']


In [433]:

list_passages = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

for i in list_passages:
    horaire= {}

    # HEURE PROCHAIN PASSAGE
    str_date = i['MonitoredVehicleJourney']['MonitoredCall']['ExpectedDepartureTime']
    date_and_time = datetime.datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S.%fZ")+datetime.timedelta(hours=1)
    str_prochain_passage = datetime.datetime.strftime(date_and_time,"%Y-%m-%d %H:%M:%S")

    # DIRECTION 
    direction = i['MonitoredVehicleJourney']['DestinationName'][0]['value']
   
    # LIGNE
    ligne = get_line_name(i['MonitoredVehicleJourney']['LineRef']['value'])
    if len(i['MonitoredVehicleJourney']['JourneyNote'])>0:
        ligne += f" - {i['MonitoredVehicleJourney']['JourneyNote'][0]['value']}"
    
    if True:
        print(f"Direction > {direction}")
        print(ligne)
        print(f"Prochain départ > {date_and_time}")
        print("*"*50)

    # ON SAUVEGARDE LES INFOS DANS UNE LISTE
    horaire.update({'arret':arret,
                    'ligne':ligne, 
                    'direction':direction, 
                    'passage':str_prochain_passage})
    infos_horaires.append(horaire)


Direction > Porte des Lilas
bus 105
Prochain départ > 2024-11-24 16:06:13
**************************************************
Direction > Porte des Lilas
bus 105
Prochain départ > 2024-11-24 16:24:36
**************************************************


In [434]:
infos_horaires

[{'arret': 'Professeur Vaillant',
  'ligne': 'bus 105',
  'direction': 'Porte des Lilas',
  'passage': '2024-11-24 16:06:13'},
 {'arret': 'Professeur Vaillant',
  'ligne': 'bus 105',
  'direction': 'Porte des Lilas',
  'passage': '2024-11-24 16:24:36'}]

SAUVEGARDE DES INFOS EN CSV

In [ ]:
with open('src/backup_messages.csv', 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=infos_messages[0].keys())
    writer.writeheader()
    writer.writerows(infos_messages)


with open('src/backup_horaires.csv', 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=infos_horaires[0].keys())
    writer.writeheader()
    writer.writerows(infos_horaires)

ESSAIS METRO

In [ ]:
mask = df_arrets['ArRType']=="metro"

mask = df_arrets['ArRName']=="Oberkampf"
df_arrets[mask]

In [ ]:
arret = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'][0]['value']
directions = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']
#
print(f"Arrêt : {arret}")
for d in directions:
    print(f"Direction : {d['MonitoredVehicleJourney']['DestinationName'][0]['value']}")
    

In [ ]:

url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'StopPointRef' : f'STIF:StopPoint:Q:{ar_ob}:'}

response = requests.get(url, headers=headers, params=p)



In [ ]:
response.content

In [ ]:
list_messages = response.json().get("Siri").get("ServiceDelivery").get('GeneralMessageDelivery')[0].get('InfoMessage')

for i in list_messages:
    print(i.get('Content').get('Message')[0].get('MessageText').get('value'))